### 1. Import lib and data

In [37]:
import pandas as pd 
import numpy as np 

In [38]:
path = '/home/hoan/Python/Code/AloNhaDat.csv'
df = pd.read_csv(path)
df.sample(3)

,city,date,direction,district,floors,front_road,id,length,price,property_legal_document,rooms,size,type_adv,type_real_estate,ward,width
45119,Hồ Chí Minh,03/11/2021,_,Quận Tân Phú,5,5m,8800491,13m,6 tỷ,Sổ hồng/ Sổ đỏ,6,52 m,Cần bán,Nhà trong hẻm,Phường Hòa Thạnh ...,4m
91482,Hồ Chí Minh,07/11/2021,_,Quận Tân Phú,2,4m,8931365,10m,"3,2 tỷ",Sổ hồng/ Sổ đỏ,2,40 m,Cần bán,Nhà trong hẻm,Phường Tân Quý ...,4m
22684,Hồ Chí Minh,22/10/2021,_,Quận Tân Bình,3,8m,9215335,10m,"7,5 tỷ",---,3,40 m,Cần bán,Nhà trong hẻm,Phường 13 ...,4m


### 2. Define processing function

In [39]:
df.columns = [column.split(' ')[0] for column in df.columns]

In [40]:
def rm_price_unit(value):

    temp = '' # luu gia tri xu ly cuoi cung (don vi: ty). Vd: 4 ty 500 trieu 12 ngan -> 4.500012
    mill_unit = {3: '.', 2: '.0', 1: '.00'} # chuyen tu trieu -> ty. Vd 2 trieu -> .002 ty
    thous_unit_01 = {3: '.000', 2: '.0000', 1: '.00000'} # chuyen tu ngan -> ty: 200 ngan -> .000200 ty
    thous_unit_02 = {3: '', 2: '0', 1:'00'}
    
    value = value.replace(',', '.') # 4,7 -> 4.7
    value = value.replace(' ', '') # 4 ty 8 trieu -> 4ty8trieu

    # 4ty8trieu12ngan -> 4.008012
    if 'tỷ' in value:
        ls_val = value.split('tỷ') # ls_val = ['4', '8trieu12ngan']
        temp = temp + ls_val[0] # tem = '4' = '' + '4'
        if 'triệu' in value:
            # '8trieu12ngan'
            ls_val = ls_val[1].split('triệu') # ls_val = ['8', '12ngan]
            # len('8') = 1 -> mill_unit = '.00' -> temp = '4' + '.00' + ''8 = 4.008
            temp = temp + mill_unit[len(ls_val[0])] + ls_val[0] 
            if 'ngàn' in value:
                # 12ngan
                ls_val = ls_val[1].split('ngàn') # ls_val = ['12', '']
                # len('12') = 2 -> thous_unit_02 = '0' -> temp = '4.008' + '0' + '12' = '4.008012'
                temp = temp + thous_unit_02[len(ls_val[0])] + ls_val[0] 
    # 980 trieu 4 ngan
    elif 'triệu' in value:
            ls_val = value.split('triệu') # ls_val = ['980', '4ngan]
            if '.' in value:
                return float(ls_val[0])
            # len('980') = 3 -> mill_unit = '.' -> temp = '' + '.' + '980' = '0.980'
            temp = temp + mill_unit[len(ls_val[0])] + ls_val[0]
            if 'ngàn' in value:
                ls_val = ls_val[1].split('ngàn') # ls_val = ['4', '']
                # len(4) = 1 -> thous_unit_02 = '00' -> temp = '.980' + '00' + '4' = '.980004'
                temp = temp + thous_unit_02[len(ls_val[0])] + ls_val[0]
    # 120 ngan
    elif 'ngàn' in value:
        ls_val = value.split('ngàn') # ls_val = ['120', '']
        # len('120') = 3 -> thous_unit_01 = '.000' -> temp = '' + '.000' + '120' = '.000120'
        temp = temp + thous_unit_01[len(ls_val[0])] + ls_val[0]
    else:
        #print(value)
        return np.nan

    temp = float(temp)
    if 'm' in value:
        temp *=-1
        
    return temp

In [41]:
ls_error = {}
def rm_measure_unit(value, column):
    if column == 'width': 
        error = {'1O': 10,4.5: 4.5,'4.5l': 4.5,'=': np.nan,'.7.1': 7.1,'tuycan': np.nan,3.75: 3.75,'4.3.': 4.3,
                'a': np.nan,5.15: 5.15,'4.2ình': 10.7,'4.2(nhà.': 4.2,'4.5x': 4.5,5.25: 5.25,'4.7=6': 5,'3.2\\6.8': 5,
                '.3.5': 3.5,'.6.6': 6.6,'4.5.': 4.5,'4.6.': 4.6,'.3.6': 3.6,'4.1+7.3': 5.7,6.0: 6,'4.9.8': 4.98,'.': np.nan,
                '3.5.': 3.5,6.5: 6.5,'4:2': 4.2,'5.5.': 5.5,4.25: 4.25,'Ã': np.nan,'4l.2': 4.2,6.8: 6.8}

    elif column == 'length':
        error = {'65-75': 70,'75;65': 70,'7p': 7,'5.×29': 5.29,'tu`ycan': np.nan,
                '1u': np.nan,'3u': np.nan,'`26': 26,'.6.3': 6.3,'`7': 7,'7/5': 7.5,
                '10/63': 10.63,'``15': 15,'2-0': 2,'1p': np.nan,'10\\': 10,'<25': 20,
                '2”': 5.08,'89?': 89,'.': np.nan,'`19': 19,'15/7': 15.7,'2r': np.nan,
                'G?n': np.nan,'13.5.': 13.5,'14.5.': 14.5,'19.5.': 19.5,'5.6.': 5.6,
                '16.5.': 16.5,'17.5.': 17.5,'10p': 10, 70.0: 70,1.0: 0, 'tuycan':np.nan}

    elif column == 'front_road':
        error = {'QL': 26, '>10': 10, '6-8': 7, '5q': 5, 'Xe': np.nan, "5'": 5, '`13': 13,'2-': 2,
                '8i': 8,'3f': 3,'`12': 12,'I': np.nan,'2-3': 2.5,'3 5': 4,'`10': 10,'6l5': 6.5,
                '16M': 16,'1"': 2.54,'6i': 6,'3.1/5': 3.2,'xe': np.nan,'12a': 12,'...': np.nan,
                '.': np.nan,'>4': 5,'ô': np.nan,'>5': 6,'Nhà': np.nan,'Qp': np.nan,'Ai': np.nan,
                7.0: 7, 2.5: 2.5, '..': np.nan, '+': np.nan,'gay': np.nan,'Xi1': np.nan,'12`': 12,
                '>3': 4,'1w': np.nan,'aâ': np.nan,'---        ': np.nan, '-': np.nan, 'H?': np.nan,
                 '5-6': 5.5, '5..': 5, 5.5: 5.5}
    elif column == 'size':
        error = {'1.000.000': 1000000,'3.500.000': 3500000,'3.400.000': 3400000,
        '20.000.000': 20000000,'12.681.000': 12681000,'KXĐ': np.nan,'10.000.000': 10000000,
        '3.060.000': 3060000,'3.118.880': 3118880,'1.141.920': 1141920,'4.280.000': 4280000,
        '4.850.000': 4850000,'100.000.000': 100000000,'120.000.000': 120000000,
        '5.800.000': 5800000,'6.888.888': 6888888}
    else:
        raise Exception('Do not support column: {}'.format(column))

    if '---' in value:
        return np.nan
    try:
        # replace [' ', "'", '`'] to ''
        value = value.replace(' ', '').replace("'","").replace('`', '')
        # replace [',', '..'] to '.'. Vd 4,7m -> 4.7m va 4..7m -> 4.7m
        value = value.replace(',', '.').replace('..', '.')
        # 4.7m
        value = value.split('m')[0] # value = ['4.7', ''][0] = 4.7
        
        # if value = 4-6 or value - 4/6 -> value = (4+6)/2 = 5 
        if '-' in value:
            ls_val = value.split('-')
            value = ( float(ls_val[0]) + float(ls_val[1]) ) / 2
        if '/' in value:
            ls_val = value.split('/')
            value = ( float(ls_val[0]) + float(ls_val[1]) ) / 2
        value = float(value)
    except:
        # if value not in error:
        #     ls_error[value] = value
        # if value la truong hop dac biet -> dung dictionary erro xu ly
        value = error[value]
        value = float(value)
    return value

### 3. Preprocessing data

In [42]:
# create a new coppy of data
df_01 = df.copy()

#### 3.1 Remove spare unit

In [43]:
df_01['price'] = df_01['price'].apply(rm_price_unit)

In [44]:
df_01['width'] = df_01['width'].apply(lambda x:rm_measure_unit(value=x,column='width'))

In [45]:
df_01['length'] = df_01['length'].apply(lambda x:rm_measure_unit(value=x,column='length'))

In [46]:
df_01['front_road'] = df_01['front_road'].apply(lambda x:rm_measure_unit(value=x,column='front_road'))

In [47]:
df_01['size'] = df_01['size'].apply(lambda x:rm_measure_unit(value=x,column='size'))

#### 3.2 Replace missing values wit nan

In [48]:
df_01['floors'] = df_01['floors'].apply(lambda x: np.nan if '---' in x else float(x))

In [49]:
df_01['rooms'] = df_01['rooms'].apply(lambda x: np.nan if '---' in x else float(x))

In [50]:
df_01['direction'] = df_01['direction'].apply(lambda x: np.nan if '_' in x else x)

In [51]:
df_01['property_legal_document'] = df_01['property_legal_document'].apply(lambda x: np.nan if '---' in x else x)

#### 3.3 Drop labels not be used

In [52]:
df_01.drop(labels=['id', 'type_adv'], axis=1, inplace=True)

In [53]:
df_01.sample(10)

,city,date,direction,district,floors,front_road,length,price,property_legal_document,rooms,size,type_real_estate,ward,width
25465,Hồ Chí Minh,26/10/2021,NaN,Quận Phú Nhuận,5.0,4.0,10.0,7.450,NaN,5.0,58.000,Nhà trong hẻm,Phường 5 ...,5.8
122491,Hồ Chí Minh,08/11/2021,NaN,Quận 10,1.0,5.0,10.0,4.700,Sổ hồng/ Sổ đỏ,2.0,30.000,Nhà trong hẻm,Phường 12 ...,3.0
72158,Hồ Chí Minh,06/11/2021,NaN,Quận Gò Vấp,2.0,3.0,8.0,2.980,Sổ hồng/ Sổ đỏ,2.0,32.000,Nhà trong hẻm,Phường 1 ...,3.5
80152,Hồ Chí Minh,06/11/2021,NaN,Quận 10,3.0,35.0,20.0,37.000,NaN,5.0,80.000,Nhà mặt tiền,Phường 12 ...,4.0
110153,Hồ Chí Minh,08/11/2021,NaN,Quận Tân Phú,NaN,NaN,NaN,8.000,NaN,NaN,256.000,Nhà trong hẻm,Phường Tân Sơn Nhì ...,NaN
19521,Hồ Chí Minh,14/10/2021,NaN,Quận 1,NaN,NaN,NaN,1900.000,NaN,NaN,1.744,Văn phòng,Phường Đa Kao ...,NaN
68209,Hồ Chí Minh,05/11/2021,Tây,Quận Gò Vấp,3.0,6.0,17.0,6.600,Sổ hồng/ Sổ đỏ,4.0,75.000,Nhà trong hẻm,Phường 9 ...,4.5
697,Bình Dương,07/10/2021,NaN,Thị xã Bến Cát,2.0,16.0,20.0,2.400,Sổ hồng/ Sổ đỏ,4.0,100.000,"Biệt thự, nhà liền kề",Phường Thới Hòa ...,5.0
670,Bình Dương,08/10/2021,NaN,Huyện Bàu Bàng,NaN,13.0,20.0,0.700,NaN,NaN,100.000,"Đất thổ cư, đất ở",Thị trấn Lai Uyên ...,5.0
3487,Bình Dương,07/11/2021,NaN,Huyện Phú Giáo,NaN,8.0,50.0,6.125,Sổ hồng/ Sổ đỏ,NaN,1.750,"Đất thổ cư, đất ở",Xã Vĩnh Hòa ...,35.0


### 4. Preprocessing categorical variables

In [54]:
for column in ['city', 'district', 'ward', 'property_legal_document', 'type_real_estate']:
    print(column)
    print(df_01[column].value_counts())
    print('----------------------------------------')
    

city
Hồ Chí Minh               111462
Đồng Nai                    9816
Bình Dương                  4217
P6 Quận 3 Hồ Chí Minh          1
Name: city, dtype: int64
----------------------------------------
district
Quận Gò Vấp                    13827
Quận Tân Bình                  12455
Quận Bình Thạnh                 9384
Quận Tân Phú                    9285
Quận 10                         6870
Quận 1                          6863
Quận Phú Nhuận                  6424
Quận Bình Tân                   5751
Quận 3                          5514
Thành phố Biên Hoà              4832
Quận 7                          4065
Thành phố Thủ Đức               3901
Quận 9                          3570
Quận 2                          3483
Quận 12                         2958
Huyện Củ Chi                    2857
Huyện Bình Chánh                2508
Quận 11                         2101
Quận 5                          2038
Huyện Nhà Bè                    1998
Quận 8                          1840
Quận 6     